In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
olympics=1
if olympics==0:
    dataset=pd.read_csv('./Data/olympics_100m.csv')
else:
    #create using scikit learn a 2 numerical features dataset with 100 samples from a linear distribution
    from sklearn.datasets import make_regression
    X,Y=make_regression(n_samples=100, n_features=2, noise=2)
    dataset=pd.DataFrame({'features':X[:,0],'features2':X[:,1],'targets':Y})
from sklearn.model_selection import train_test_split
dataset_train,dataset_test=train_test_split(dataset,test_size=0.2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
#plot
if dataset_train.shape[1]==2:
    dataset_train.plot.scatter(x='features',y='targets')

In [27]:
#coarse to fine approach
from ridge import Ridge_Regression
X=dataset_train.drop('targets',axis=1).values.reshape(-1,dataset_train.shape[1]-1)
Y=dataset_train['targets'].values.reshape(-1,1)
## given the low amount of data available we decided to do a K-fold cross validation
## the parameter lambda follows a coarse to fine approach
## the coarse search is done with a step of 0.1 and the fine search with a step of 0.01
k=len(X)
fold_size=int(len(X)/k)
lambda_mses=[]
# generate the intervals for the coarse to fine approach from 1 to 10^-20
lambda_values=np.array(10.0 ** np.arange(10, -10, -1))
for lambda_value in lambda_values:
    print('Lambda value: ',lambda_value)
    ridge=Ridge_Regression(lambda_value)
    mses=[]
    for i in range(k):
        # K-fold cross validation (take the data from the i-th fold as test data)
        Xi=np.concatenate((X[:i*fold_size],X[(i+1)*fold_size:]),axis=0)
        Yi=np.concatenate((Y[:i*fold_size],Y[(i+1)*fold_size:]),axis=0)
        ridge.Train(Xi,Yi)
        y_pred=ridge.Predict(X[i*fold_size:(i+1)*fold_size])
        mses.append(((Y[i*fold_size:(i+1)*fold_size]-y_pred)**2).mean())
    lambda_mses.append(np.mean(mses))
    print('lambda: ',lambda_value,' mse: ',np.mean(mses))
print('best lambda atm: ',lambda_values[np.argmin(lambda_mses)])
lambda_value=lambda_values[np.argmin(lambda_mses)]

Lambda value:  10000000000.0
lambda:  10000000000.0  mse:  3905.514716556719
Lambda value:  1000000000.0
lambda:  1000000000.0  mse:  3905.5141698212974
Lambda value:  100000000.0
lambda:  100000000.0  mse:  3905.5087024733984
Lambda value:  10000000.0
lambda:  10000000.0  mse:  3905.454029626441
Lambda value:  1000000.0
lambda:  1000000.0  mse:  3904.9073643528527
Lambda value:  100000.0
lambda:  100000.0  mse:  3899.447024669794
Lambda value:  10000.0
lambda:  10000.0  mse:  3845.46845009307
Lambda value:  1000.0
lambda:  1000.0  mse:  3362.258850276978
Lambda value:  100.0
lambda:  100.0  mse:  1237.3548485653964
Lambda value:  10.0
lambda:  10.0  mse:  54.53449523706928
Lambda value:  1.0
lambda:  1.0  mse:  4.6653234347792605
Lambda value:  0.1
lambda:  0.1  mse:  4.051409392308111
Lambda value:  0.01
lambda:  0.01  mse:  4.0457897424698634
Lambda value:  0.001
lambda:  0.001  mse:  4.04579990943745
Lambda value:  0.0001
lambda:  0.0001  mse:  4.045806662146487
Lambda value:  1e-0

In [28]:
order_of_magnitude=lambda_value
for order in np.array(10.0 ** np.arange(np.log10(order_of_magnitude)+1, -10, -1)):
    lambda_values=np.linspace(max(lambda_values[np.argmin(lambda_mses)]-1*order,0),lambda_values[np.argmin(lambda_mses)]+1*order,num=(20+1))
    lambda_mses=[]
    for lambda_value in lambda_values:
        print('lambda_value: ',lambda_value)
        ridge=Ridge_Regression(lambda_value)
        mses=[]
        for i in range(k):
            # K-fold cross validation (take the data from the i-th fold as test data)
            Xi=np.concatenate((X[:i*fold_size],X[(i+1)*fold_size:]),axis=0)
            Yi=np.concatenate((Y[:i*fold_size],Y[(i+1)*fold_size:]),axis=0)
            ridge.Train(Xi,Yi)
            y_pred=ridge.Predict(X[i*fold_size:(i+1)*fold_size])
            mses.append(((Y[i*fold_size:(i+1)*fold_size]-y_pred)**2).mean())
        lambda_mses.append(np.mean(mses))
        print('lambda: ',lambda_value,' mse: ',np.mean(lambda_mses))
    print('best lambda atm: ',lambda_values[np.argmin(lambda_mses)])
    lambda_value=lambda_values[np.argmin(lambda_mses)]

print('best lambda: ',lambda_value)

lambda_value:  0.0
lambda:  0.0  mse:  4.045807476841286
lambda_value:  0.0055
lambda:  0.0055  mse:  4.045794634105847
lambda_value:  0.011
lambda:  0.011  mse:  4.045794772313462
lambda_value:  0.0165
lambda:  0.0165  mse:  4.045807887338482
lambda_value:  0.022
lambda:  0.022  mse:  4.045833975056647
lambda_value:  0.027499999999999997
lambda:  0.027499999999999997  mse:  4.0458730313451055
lambda_value:  0.033
lambda:  0.033  mse:  4.045925052082399
lambda_value:  0.0385
lambda:  0.0385  mse:  4.045990033148467
lambda_value:  0.044
lambda:  0.044  mse:  4.046067970424644
lambda_value:  0.049499999999999995
lambda:  0.049499999999999995  mse:  4.04615885979366
lambda_value:  0.05499999999999999
lambda:  0.05499999999999999  mse:  4.0462626971396425
lambda_value:  0.0605
lambda:  0.0605  mse:  4.046379478348112
lambda_value:  0.066
lambda:  0.066  mse:  4.046509199305986
lambda_value:  0.0715
lambda:  0.0715  mse:  4.046651855901573
lambda_value:  0.077
lambda:  0.077  mse:  4.046807

In [29]:
ridge=Ridge_Regression(lambda_value)
ridge.Train(X,Y) ## train on the training dataset
X_test=dataset_test.drop('targets',axis=1).values.reshape(-1,dataset_test.shape[1]-1)
Y_test=dataset_test['targets'].values.reshape(-1,1)
y_pred=ridge.Predict(X_test)
from sklearn.linear_model import Ridge
ridge_sklearn=Ridge(alpha=lambda_value)
ridge_sklearn.fit(X,Y)
y_pred_sklearn=ridge_sklearn.predict(X_test)
for i in range(len(X_test)):
    print('year: ',X_test[i],' time: ',Y_test[i])
for i in range(len(X_test)):
    print('year: ',X_test[i],' time: ',y_pred[i])
    print('scikit: year: ',X_test[i],' time: ',y_pred_sklearn[i])
    
print('w_mio: ',ridge.__str__())
print('w_sklearn: ',ridge_sklearn.coef_,ridge_sklearn.intercept_)
# mse on the training set
print('mse: ',((Y_test-ridge.Predict(X_test))**2).mean())
print('mse sklearn: ',((Y_test-ridge_sklearn.predict(X_test))**2).mean())

year:  [0.72693505 0.92755344]  time:  [59.27586391]
year:  [-0.66809989  0.49407653]  time:  [-29.74169314]
year:  [-1.66072939  1.5655845 ]  time:  [-77.78500753]
year:  [-0.11362983  0.86291347]  time:  [4.09770932]
year:  [ 2.36731892 -0.09962141]  time:  [138.96871699]
year:  [ 0.54020092 -0.15879982]  time:  [31.22590734]
year:  [-0.85874296 -2.38545943]  time:  [-88.06370722]
year:  [-0.06657106  0.80165505]  time:  [9.68968667]
year:  [-1.46860866 -0.23984708]  time:  [-90.75665417]
year:  [1.57408356 1.09109551]  time:  [112.15437296]
year:  [0.68602157 1.74823395]  time:  [69.7164357]
year:  [-1.43857814  0.58900385]  time:  [-78.16848215]
year:  [ 0.82264623 -1.15596841]  time:  [30.02584732]
year:  [-1.52618211 -1.84094637]  time:  [-115.96617029]
year:  [ 0.40882239 -1.05020275]  time:  [9.22866897]
year:  [-0.4668186  -1.27536637]  time:  [-45.83480622]
year:  [-0.0331832   1.07217352]  time:  [15.30589172]
year:  [0.02170656 0.55158547]  time:  [13.66096877]
year:  [0.54

In [30]:
#plot if 1D
if dataset_train.shape[1]==2:
    import matplotlib.pyplot as plt   
    ridge=Ridge_Regression(lambda_value)
    ridge.Train(X,Y)
    y_pred=ridge.Predict(X)
    from sklearn.linear_model import Ridge
    ridge_sklearn=Ridge(alpha=lambda_value)
    ridge_sklearn.fit(X,Y)
    y_pred_sklearn=ridge_sklearn.predict(X)
    from sklearn.linear_model import RidgeCV
    ridge_cv=RidgeCV(cv=k)
    ridge_cv.fit(X,Y)
    y_pred_cv=ridge_cv.predict(X)
    plt.scatter(X, Y, label='Training Points')
    plt.plot(X, y_pred, label='My Ridge Regression', color='red')
    plt.plot(X, y_pred_sklearn, label='Scikit-learn Ridge Regression', linestyle='--', color='green')
    
    plt.title('Ridge Regression Comparison')
    plt.xlabel('Year')
    plt.ylabel('Time')
    plt.legend()
    plt.show()